3. データ準備と前処理（詳説版）
① データ収集（Data Collection）
▶ ウェブスクレイピング（Web Scraping）
Web上のデータを自動的に収集する方法。
PythonではrequestsとBeautifulSoupをよく使います。

In [1]:
# PythonでのWebスクレイピング例
import requests
from bs4 import BeautifulSoup

url = 'https://qiita.com/sivertigo/items/8f61f02f7c84b786697a'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# ページ内のすべての段落<p>を取得
paragraphs = soup.find_all('p')
for p in paragraphs:
    print(p.text)


Go to list of users who liked
Share on X(Twitter)
Share on Facebook
Add to Hatena Bookmark
Delete article
Deleted articles cannot be recovered.
Draft of this article would be also deleted.
Are you sure you want to delete this article?
More than 3 years have passed since last update.
ずっとJava11を使っていました。そのためすっかりラムダ式やStreamAPIでの書き方に慣れていたのですが、最近Java7を使って稼働しているプロジェクトに異動したので、何ができて何ができないのか、ちょっと調べてみました。
私見ですが主要だと思う仕様変更を並べてみました。アプリ実装に関係する部分のみピックアップしています。
このバージョンのこれは入れておいたほうが良いよ！との意見がありましたらコメントにぜひお願いします。

2014/03/18 正式リリース(GA)
https://openjdk.java.net/projects/jdk8/
https://openjdk.java.net/projects/lambda/
https://qiita.com/sano1202/items/64593e8e981e8d6439d3
無名関数を簡潔に呼び出せる。
StreamAPIと合わせてすごくよく使う。
Java Stream APIをいまさら入門 - Qiita 
コレクション(から作ったStream)に対して中間処理、終端処理を実行できる。
メソッドチェーンで様々な処理を簡潔に書くことができてよい。



nullを許容し、nullチェックの仕組みを持ったクラス。
https://openjdk.java.net/jeps/150
DateTimeFormatterとかZonedDateTimeとか、日時を扱うのに便利なクラスがたくさん。
2017/09/21 正式リリース(GA)
https://openjdk.java.net/projects/jdk9/
なんとJava8から3年半も経ってい

▶ オープンデータセット（Open Dataset）
Kaggle、UCIリポジトリなどの公開データセットを利用。

In [2]:
# sklearnライブラリによるオープンデータセットの取得例
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target
print(df.head())


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   species  
0        0  
1        0  
2        0  
3        0  
4        0  


▶ API経由のデータ取得（API Access）
REST APIを通じて外部サービスからデータを取得する方法。

In [2]:
# OpenWeatherMap APIの例 https://home.openweathermap.org/api_keysでKeyを取得
import requests

# 天気データから主要情報を抽出
from googletrans import Translator
import time

def translate_weather_to_japanese(weather_data):
    """天気データを日本語に翻訳して主要情報を抽出する関数"""
    translator = Translator()
    
    # 温度をケルビンから摂氏に変換
    temp_celsius = weather_data['main']['temp'] - 273.15
    feels_like_celsius = weather_data['main']['feels_like'] - 273.15
    
    # 天気の説明を日本語に翻訳（エラーハンドリング付き）
    weather_desc_en = weather_data['weather'][0]['description']
    try:
        # 少し待機してから翻訳実行
        time.sleep(0.1)
        translation_result = translator.translate(weather_desc_en, src='en', dest='ja')
        weather_desc_jp = translation_result.text
    except Exception as e:
        print(f"翻訳エラー: {e}")
        # フォールバック：簡単な辞書翻訳
        weather_translations = {
            'clear sky': '快晴',
            'few clouds': '晴れ時々曇り',
            'scattered clouds': '曇り',
            'broken clouds': '曇り',
            'overcast clouds': '曇り空',
            'shower rain': 'にわか雨',
            'rain': '雨',
            'thunderstorm': '雷雨',
            'snow': '雪',
            'mist': '霧'
        }
        weather_desc_jp = weather_translations.get(weather_desc_en, weather_desc_en)
    
    # 主要情報を辞書形式で整理
    weather_info = {
        '都市名': weather_data['name'],
        '国': weather_data['sys']['country'],
        '天気（英語）': weather_desc_en,
        '天気（日本語）': weather_desc_jp,
        '現在気温': f"{temp_celsius:.1f}°C",
        '体感気温': f"{feels_like_celsius:.1f}°C",
        '湿度': f"{weather_data['main']['humidity']}%",
        '気圧': f"{weather_data['main']['pressure']} hPa",
        '風速': f"{weather_data['wind']['speed']} m/s",
        '風向': f"{weather_data['wind']['deg']}°",
        '雲量': f"{weather_data['clouds']['all']}%",
        '視界': f"{weather_data['visibility']/1000:.1f} km"
    }
    
    return weather_info


API_KEY = '481bf0529fcc0c9d245acc818c17a874'
city = 'Tokyo'
url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}'

response = requests.get(url)
data = response.json()
print(data)

# データを抽出して表示
weather_summary = translate_weather_to_japanese(data)
print(f"=== {city}の天気情報 ===")
for key, value in weather_summary.items():
    print(f"{key}: {value}")

{'coord': {'lon': 139.6917, 'lat': 35.6895}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 305.25, 'feels_like': 309, 'temp_min': 305.25, 'temp_max': 305.25, 'pressure': 1002, 'humidity': 55, 'sea_level': 1002, 'grnd_level': 1001}, 'visibility': 10000, 'wind': {'speed': 7.44, 'deg': 179, 'gust': 6.7}, 'clouds': {'all': 100}, 'dt': 1753690624, 'sys': {'country': 'JP', 'sunrise': 1753645545, 'sunset': 1753696168}, 'timezone': 32400, 'id': 1850144, 'name': 'Tokyo', 'cod': 200}
翻訳エラー: 'coroutine' object has no attribute 'text'
=== Tokyoの天気情報 ===
都市名: Tokyo
国: JP
天気（英語）: overcast clouds
天気（日本語）: 曇り空
現在気温: 32.1°C
体感気温: 35.9°C
湿度: 55%
気圧: 1002 hPa
風速: 7.44 m/s
風向: 179°
雲量: 100%
視界: 10.0 km


/tmp/ipykernel_44496/4159330745.py:68: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  weather_summary = translate_weather_to_japanese(data)
